In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'Open_2'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df1 = pd.read_csv('datasets/OpenAccessArDialect/Sentiment_Anaysis.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3
df2 = pd.read_csv('datasets/OpenAccessArDialect/dialect.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3

df2 = df2[df2['dialect'] == 'Morocco']
      
display(df1.columns)
display(df1[:4])

display(df2.columns)
display(df2[:4])
display(len(df2))



df = df1[df1['Twits'].isin(df2['Twits'])]

display(df[:5])
display(len(df))

c = df['label'].value_counts()
display(c)

classes = set(df['label'].values)
display(classes)

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

df = df[['Twits', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))



max_sequence_length = 128



models = [ 
        'faisalq/EgyBERT',            
    'faisalq/SaudiBERT',            
    'tunis-ai/TunBERT',
    'alger-ia/dziribert',
    'SI2M-Lab/DarijaBERT',
    'otmangi/MorRoBERTa',
    'otmangi/MorrBERT'
            
]





seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
            
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['Twits'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True)
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 12
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 50, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 50
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-18 15:01:49.541536: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 15:01:49.728668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 15:01:50.395873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Index(['Unnamed: 0', 'Twits', 'label'], dtype='object')

,Unnamed: 0,Twits,label
0,0,what happens,neu
1,1,😞😞,neg
2,2,ان شاء الله,neu
3,3,بالتوفيق ان شاء الله,pos


Index(['Twits', 'dialect'], dtype='object')

,Twits,dialect
13393,slawiya msikina tb9a f darha 😐😕😕,Morocco
13394,ناس د الرباط أش واقع عندكم تما ... ياكما شي شبه عائلية 😐,Morocco
13395,لي دوا مشاا 😂😂,Morocco
13396,: وزراء تقلقوا,Morocco


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at otmangi/MorrBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6816 [00:00<?, ? examples/s]

Map:   0%|          | 0/1704 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
50,0.398700,0.310427,0.880869,0.675646
100,0.297500,0.298319,0.896127,0.709025
150,0.224700,0.319014,0.875000,0.697351
200,0.208000,0.297875,0.894366,0.724290
250,0.107900,0.407760,0.889671,0.734273
300,0.086900,0.415330,0.886150,0.691722
350,0.073600,0.485916,0.879108,0.691090
400,0.042900,0.582877,0.884390,0.707364
450,0.040200,0.549538,0.890258,0.724016
500,0.028800,0.572886,0.886150,0.712613


,Model,Accuracy,F1
0,SI2M-Lab/DarijaBERT,0.896127,0.762304
3,alger-ia/dziribert,0.889085,0.747426
6,faisalq/EgyBERT,0.893192,0.736059
9,faisalq/SaudiBERT,0.901408,0.759349
12,otmangi/MorRoBERTa,0.886737,0.744082
15,otmangi/MorrBERT,0.889671,0.734273
18,tunis-ai/TunBERT,0.879108,0.682487
